## Traffic sign detection and classification (with deep learning)

In [6]:
from pathlib import Path

import xml.etree.ElementTree as ET

from torch.utils.data import Dataset, DataLoader

import pandas as pd
from pandas import DataFrame


#### Loading Data

In [14]:
imgs_path_name = './dataset/images'
anno_path_name = './dataset/annotations'
train_names_path = Path('./dataset/train.txt')
test_names_path = Path('./dataset/test.txt')

# Gets train or test data file names


def train_test_file_names(train_test_data_path: Path):
    file_names = []
    with open(train_test_data_path, 'r') as file:
        file_names = file.readlines()
        file_names = [file_name.rstrip() for file_name in file_names]
    return file_names

# Generates pandas dataframe for train dataset


def generate_train_df(train_file_names: list, only_biggest_area_object: bool):
    anno_list = []
    for train_file_name in train_file_names:
        anno_xml_root = ET.parse(
            Path(anno_path_name + "/" + train_file_name + ".xml"))
        anno = {}
        anno['img_file_path'] = Path(
            imgs_path_name + "/" + anno_xml_root.find("filename").text)
        anno['width'] = anno_xml_root.find("size/width").text
        anno['height'] = anno_xml_root.find("size/height").text
        anno['class'] = anno_xml_root.find("object/name").text

        xml_objects = anno_xml_root.findall("object")
        if only_biggest_area_object:
            max_area = 0
            max_area_obj = {}
            for xml_object in xml_objects:
                curr_obj = {}
                curr_obj["xmin"] = int(xml_object.find("bndbox/xmin").text)
                curr_obj["ymin"] = int(xml_object.find("bndbox/ymin").text)
                curr_obj["xmax"] = int(xml_object.find("bndbox/xmax").text)
                curr_obj["ymax"] = int(xml_object.find("bndbox/ymax").text)
                curr_obj_area = (curr_obj["xmax"] - curr_obj["xmin"]) * (curr_obj["ymax"] - curr_obj["ymin"])
                if curr_obj_area > max_area:
                    max_area_obj = curr_obj
                    max_area = curr_obj_area

            anno["xmin"] = max_area_obj["xmin"]
            anno["ymin"] = max_area_obj["ymin"]
            anno["xmax"] = max_area_obj["xmax"]
            anno["ymax"] = max_area_obj["ymax"]

        anno_list.append(anno)

    return pd.DataFrame(anno_list)
    


In [15]:
print(display(generate_train_df(
    train_test_file_names(train_names_path), True).iloc[603]))


3721
1666


img_file_path    dataset/images/road327.png
width                                   300
height                                  400
class                            speedlimit
xmin                                     92
ymin                                    173
xmax                                    153
ymax                                    234
Name: 603, dtype: object

None


In [9]:
import os
import os.path
import xml.etree.ElementTree as ET

xml_files = [name for name in os.listdir('./dataset/annotations')]
validation_dictionary = {}

for name in xml_files:
    # Parse the xml
    mytree = ET.parse('./dataset/annotations/' + name)

    # Get number from name
    order = ''.join(i for i in name if i.isdigit())

    # Get sign types
    validation_dictionary[order] = mytree.getroot().find(
        'object').find('name').text


In [10]:
sign_types = []
for entry in validation_dictionary:
    if validation_dictionary[entry] not in sign_types:
        sign_types.append(validation_dictionary[entry])
print(sign_types)


['trafficlight', 'speedlimit', 'stop', 'crosswalk']
